# Pythonのオブジェクトについて

Pythonはすべてオブジェクトです

In [ ]:
# 変数が持つのはオブジェクトのIDです
x = 1
id(x)

値はPython実行環境において、オブジェクト管理空間に生成・格納されています。

|参照オブジェクトID|オブジェクト|
|---|---|
|0x12345678|int(1)|
|0x23456789|str('abc')|

変数はオブジェクトへの参照を持っています。

|変数名|参照オブジェクトID|
|---|---|
|a|0x12345678|
|b|0x23456789|

変数名とオブジェクトIDのテーブル → シンボルテーブル

* オブジェクト格納領域は実行環境内では**グローバルに**確保されています <-- 超重要※1
* 変数名と参照オブジェクトIDの対応は実行環境内のシンボルテーブルで管理されています
    * シンボルテーブルはコード内でシンボルが登場したときに参照されます
        * 作成する時は既存の名前がないかを確認し、オブジェクト作成→ID取得→シンボルテーブルに登録
        * 参照するときはシンボルテーブル内に名前が有るかを確認してIDを取得する
    * 参照し、シンボルテーブル上に名前がない時は「未定義のシンボル」としてエラーになる
* シンボルテーブルへのマッピングはPythonのランタイム起動時から行われている
    * 標準的に利用可能な関数(printとか)も登録
    * よく使う(と考えられている)数値(整数)も登録
    * 予約語以外のものは全てテーブルにある
        * 悪く使うと <del>書き換える</del>置き換えることすらできる

In [ ]:
x = 1
y = 2
print(f"x: {id(x)}, y: {id(y)}")
y = 1 # 変更するとIDが変わります
print(f"x: {id(x)}, y: {id(y)}") # 同じ値なのでIDも同一

# リストはリスト自体がIDを持ちます
x = ["foo", "bar", "baz"]
print(f"x: {id(x)}")
print(f"x[0]: {id(x[0])}")
print(f"'foo': {id('foo')}")

ただし、生成時に同じものでないとIDは別になります
* 同値性: 値の内容が同じ(`==`)
* 同一性: 変数の指すIDが同じ(`is`)
Python的には同一性の方がチェックが速い(IDわかれば良いだけなので)

In [ ]:
x = "hogeStr"
y = "hoge".__add__("Str")
print(f"x=>{x} y=>{y}") # 各値を確認
print(f"{x == y}, {x is y}") # 同値性と同一性を確認
print(f"{id(x)}, {id(y)}") # 同じ文字だけど代入時の差違により別ID


* シンボルテーブルに入っている内容は`globals()`で参照可能
* 組み込み関数は `__builtins__` モジュールに入っている
    `print`の実体は`__builtins__.print`

In [ ]:
# globals()の中身を確認
for k, v in globals().items():
    print(f"{k} => {v}")

In [ ]:
# __builtins__の中身も確認
print(dir(__builtins__))

In [ ]:
# このことからIDを奪えば別の名前で関数を呼べるし
backup_print = print
backup_print("fugahoge")

# うっかりリスト作成を破壊するようなこともできます
list = __builtins__.list # 繰り返し実行用
backup_list = list

x = list((1,2,3))
print(x)
list = [3,4,5]
print(list)
#  y = list((1,2,3)) # ここでエラーが発生します


関数(`def`)は、内部的なスコープを作成し、globalスコープと別のスコープを持ちます。
* localスコープ
    * `locals()`で参照可能
* ローカルスコープ内の変数は関数内でのみ有効
    * シンボル名がグローバルと被った場合もローカル側が優先される(local->global)
* global宣言をすることで、グローバルスコープの変数を参照できる

In [ ]:
x = 42
y = 5

def foo(x):
    print("---")
    print(locals())
    print("---")
    # locals()にない→globals()から取得(できなければエラー)
    print(y)

foo(10)

関数の戻り値はどうなるのか?
* 関数内で生成されたオブジェクトは、関数のスコープが終了すると消滅します
* 関数の戻り値は、その関数内で生成されたオブジェクトのIDを返します
* オブジェクトのID自体はグローバルで管理されているのでシンボルとのつながりは消えても参照は可能 → ※1

In [ ]:
def foo():
    ret = 42
    print(f"id(ret) = {id(ret)} in foo()")
    return ret

x = foo()
print(f"id(x) = {id(x)} in main()")

この辺りを理解していると、Pythonの挙動を後から差し替える邪悪な行為が可能だし、関数も所詮はID管理なので値として返せてしまいます(ファーストクラスオブジェクト)

In [ ]:
def foo():
    def inner(): # 内部関数
        print("inner")
    return inner

x = foo()
x() # inner

この仕組みを理解すると、Pythonのスコープの理解が進むと同時に、
既存の関数の前後に処理を追加するという邪悪な行為が可能になります(デコレータの原理)。

In [ ]:
# 既存の関数をラップする形で関数呼び出しの前にメッセージを追加する例

def before(function):
    outer = __builtins__.print
    if function.__name__ == 'wrapper': # 二重ラップ防止策
        return function
    # ラップする関数を定義
    def wrapper(*args, **kwargs):
        outer(f"{function.__name__}が呼ばれました")
        return function(*args, **kwargs)
    return wrapper

print = before(print)
print("hoge")